In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents.initialize import initialize_agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from langchain_google_genai import GoogleGenerativeAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

/home/cyrus/dev/llamp/api/src/llamp/mp/tools.py:231: SyntaxWarning: invalid escape sequence '\s'
  """useful when you need dielectric properties like dielectric constant,


No module named 'phonopy'
No module named 'phonopy'


In [2]:
top_llm = GoogleGenerativeAI(
    temperature=0,
    model="models/gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = GoogleGenerativeAI(
    temperature=0.7,
    model="models/gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    max_retries=5,
    # streaming=True,
    # callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool()]
tools += [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    # ArxivAgent(llm=bottom_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, Wikipedia, and a python 
    REPL, which you can use to execute python code. If you get an error, debug 
    your code and try again. Only use the output of your code to answer the 
    question. Ask user to clarify their queries if needed. Please note that you 
    don't have direct control over MP but through multiple assistant agents to 
    help you. You need to provide complete context in the input for assistants to 
    do their job.
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)


In [3]:
agent_executor.invoke(
    {
        "input": "Please give me the mass density (in g/cm^3) and volume per atom (in Angstrom^3) of stable NaCl.",
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: Get summary info for material NaCl

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "NaCl"
  }
}
```Error on search_materials_summary__get: `fields` must be specified in the query. Please provide more detailed reqeust arguments or try smaller request by specifying 'limit' in request.Action:
```
{
  "action": "Final Answer",
  "action_input": "Please provide more detailed reqeust arguments or try smaller request by specifying 'limit' in request."
}

> Finished chain.
{'input': 'Get summary info for material NaCl', 'output': "Please provide more detailed reqeust arguments or try smaller request by specifying 'limit' in request."} Do I need to use a tool? Yes
Action: MPStructureRetriever
Action Input: Get structure for stable NaCl

> Entering new AgentExecutor chain...
Action:
```
{
  "action

{'input': 'Please give me the mass density (in g/cm^3) and volume per atom (in Angstrom^3) of stable NaCl.',
 'chat_history': [],
 'output': 'I do not have access to the data requested.'}

In [4]:

llm_gemini = GoogleGenerativeAI(
    temperature=0,
    model="models/gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()],
)

llm_gemini.invoke(agent_executor.memory.chat_memory.messages[0].content)


'**Mass density:** 2.165 g/cm^3\n\n**Volume per atom:** 23.9 Angstrom^3'